<a href="https://colab.research.google.com/github/chunkyshrapnel/Oregon_Project/blob/main/oregon_projv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=FVozMHiBN1ZdxWaAxJbxhojdGbw4z4HczNqQmX-CKrQ&tc=Q8wLFGfyvfYSV8sRxKiwqheGrc0ruVvFR1tc2WzvnE8&cc=UIlgvUu1vcjOnU_h4dr5_M2aXAPshaICXHrMdsriiqs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhVOoCtTva4J7ttDwnLwAUpuKUNRt5qiMvfDbhwnbQRqY-vr4eYqsLM

Successfully saved authorization token.


In [ ]:
# This will be the UI
import time
import ee
#ee.Authenticate()
ee.Initialize()

def main():

  # Set default values for exporting
  start_date = '2001-03-01'
  end_date = '2001-04-01'
  model = 'disalexi'
  boundary_name, boundary = get_boundary_geometry()
  image_coll = ee.ImageCollection\
   ('projects/openet/disalexi/conus/gridmet/monthly/provisional').\
   filterDate(start_date, end_date)

  # Loop runs until an export call is made.
  # Everytime a value is changed, the menu is displayed again.
  while(True):

    # Dispaly menu to console
    print_menu(model, start_date, end_date, boundary_name)

    # Get user input and pass on to match statement
    selected_menu_option = input()

    match selected_menu_option:

      case 'y':

        # all images are mosaiced and exported
        export_handler(image_coll,start_date, end_date, boundary, model)

        # Terminate loop and end program
        exit();

      case '1':

        print_model_menu();
        model, image_coll = change_model(start_date, end_date);

      case '2':

        print("Enter start date in form: yyyy-mm-dd")

      case '3':

        print("Enter end date in form: yyyy-mm-dd")

      case _:

          print('ERROR: Invalid Input')


def print_menu(model, start_date, end_date, boundary):

  print("\nEnter a number to change the export option.\n")
  print("1 - MODEL: ", model)
  print("2 - START DATE: ", start_date)
  print("3 - END DATE: ", end_date)
  print("4 - Boundary: ", boundary + "(NOT CHANGEABLE AT THIS TIME)")
  print("\nEnter 'y' to export image collection.")


# This function is a place holder for when the user can pic their own geometry
def get_boundary_geometry():

  states = ee.FeatureCollection('TIGER/2018/States');
  boundary = states.filter(ee.Filter.eq('GEOID', '41'));

  return "Oregon State Boundary", boundary


# This function mosaics images and initaites exporting all images
def export_handler(image_coll,start_date, end_date, boundary, model):

      image_coll = mosaci_all_images(image_coll)

      for i in range(image_coll.length().getInfo()):

        export_tif(image_coll.get(i),start_date, end_date, boundary, model);


def print_model_menu():

        print("\nEnter a number to change the export option.\n")
        print("1 - DisALEXI")
        print("2 - eeMETRIC")
        print("3 - geeSEBAL")
        print("4 - PT-JPL")
        print("5 - SIMS ")
        print("6 - SSEBop")
        print("7 - Ensemble")


def change_model(start_date, end_date):

      user_input = input()

      match user_input:

        case '1':

            return 'disalexi', ee.ImageCollection\
                              ('projects/openet/disalexi/conus/gridmet/monthly/provisional').\
                              filterDate(start_date, end_date)
        case '2':

            return 'eemetric', ee.ImageCollection\
                            ('projects/openet/eemetric/conus/gridmet/monthly/provisional')\
                            .filterDate(start_date, end_date)
        case '3':

            return 'geesebal', ee.ImageCollection\
                            ('projects/openet/geesebal/conus/gridmet/monthly/provisional')\
                            .filterDate(start_date, end_date)
        case '4':

            return 'ptjpl', ee.ImageCollection\
                            ('projects/openet/ptjpl/conus/gridmet/monthly/provisional')\
                            .filterDate(start_date, end_date)
        case '5':

            return 'sims', ee.ImageCollection\
                            ('projects/openet/sims/conus/gridmet/monthly/provisional')\
                            .filterDate(start_date, end_date)
        case '6':

            return 'ssebop', ee.ImageCollection\
                            ('projects/openet/ssebop/conus/gridmet/monthly/provisional')\
                            .filterDate(start_date, end_date)
        case '7':

            return 'ensemble', ee.ImageCollection\
                            ('projects/openet/ensemble/conus/gridmet/monthly/provisional')\
                            .filterDate(start_date, end_date);
        case _:
            print('ERROR: Invalid Input')


# This is the helper function that performs necessary maniplation and
# calls the fucntion that mosaics the 10T and 11T pairs.
def mosaci_all_images(image_coll):

  # Remove all images that are not in 10T or 11T
  image_coll = image_coll.filter(ee.Filter.inList('mgrs_tile', ['10T','11T']))

  # hacky list manipulation.
  # Convert to list and get size of list/2
  to_list = image_coll.toList(image_coll.size());
  size_of_sub_list = to_list.size().divide(2);

  # Splits the list into 2 listst where all the 10T and 11T images have theeir own list
  ten_t_list = to_list.splice(size_of_sub_list, size_of_sub_list)       # EPSG:32610 T10
  eleven_t_list = to_list.splice(0, size_of_sub_list)                   # EPSG:32611 T11

  # combine the two sepearte lists into one list of list pairs
  combinations = ten_t_list.zip(eleven_t_list);

  return combinations.map(mosaic_image_pairs)


# Takes each pair of images(10T and 11T) and mosaics them.
def mosaic_image_pairs(lst):

  lst = ee.List(lst);

  my_id = ee.Image(lst.get(0)).id().slice(4);

  # crs: EPSG:4326 (not real?)
  new_mosaic = ee.Image(ee.ImageCollection([lst.get(0), lst.get(1)]).mosaic()).set("system:index",my_id)

  return new_mosaic;


def export_tif(img,start_date, end_date, boundary, model):

  # clip so that idahoe and washington data is left out
  img = ee.Image(img);
  img = img.clip(boundary.geometry());

  # convert all bands to unsigned int16. If types are not consistent than an error is thrown.
  img = img.toUint16();

  # Create file path
  path = 'oregon/' + model + '_' + start_date + '_' + end_date + '/' + img.id().getInfo()

  # Export to Google bucket as a .tif file. (GeoTIFF)
  task = ee.batch.Export.image.toCloudStorage(**{
    'image': img,
    #description: path,
    'bucket': 'openet_temp',
    'fileNamePrefix': path,
    'crs': img.projection(),
    #'crsTransform': [30, 0, 239985, 0, -30, 5299995],
    'scale': 30,
    'region': boundary.geometry(),
    'maxPixels': 1e13,
    'fileDimensions' : 51200

  })

  task.start()

  while task.active():
    print('Polling for task (id: {0}) - Image:{1}.'.format(task.id, img.id().getInfo()))
    time.sleep(15)


main()



Enter a number to change the export option.

1 - MODEL:  disalexi
2 - START DATE:  2001-03-01
3 - END DATE:  2001-04-01
4 - Boundary:  Oregon State Boundary(NOT CHANGEABLE AT THIS TIME)

Enter 'y' to export image collection.
